# Species Richness by Location Map

This notebook generates and exports an HTML format interactive map of species richness at reported locations. To use it, update the country and, optionally, the year in the first code cell, below, before running the notebook.

If the year is specified, only locations reported in that year will be included.

If the year is left blank, all locations reported in the specified country for all time will be included.

In [ ]:
# Year to report on. Optional but if specified, should be in YYYY format e.g. 2025
year = ""

# Country to report on
country = ""

# Initial Zoom Level for the map
zoom = 6

In [ ]:
from pathlib import Path
import sqlparse

# Select the query file based on whether the year is specified
query_file_name = "richness_by_year.sql" if year else "richness.sql"

# Read the query file
query_file_path = Path("sql") / query_file_name
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Replace the country and year placeholders
query = query.replace("$YEAR", year) \
             .replace("$COUNTRY", country)

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case='upper'))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["NATURE_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found for country '{country}'"
    if year:
        message += f" during '{year}'"
    raise ValueError(message)

In [ ]:
# Make sure the latitude and longitude have been read as numbers, not strings
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

# Calculate the data for the richness chart
richness = (
    df.groupby(["Location", "Latitude", "Longitude"])["Species"]
    .nunique()
    .reset_index()
)

richness.columns = ["Location", "Latitude", "Longitude", "Richness"]

# Print the richness data
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(richness)

In [ ]:
import re

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Export the richness data to Excel
clean_country = re.sub("[^0-9a-zA-Z ]+", "", country).replace(" ", "-")
export_file_name = f"{year}-{clean_country}-Richness" if year else f"{clean_country}-Richness"
export_file_path = export_folder_path / f"{export_file_name}.xlsx"
richness.to_excel(export_file_path.absolute(), sheet_name="Location Richness")

In [ ]:
import folium
from folium import CircleMarker
from folium.plugins import MarkerCluster

# Center map on the average lat/lon of all your locations
richness_map = folium.Map(
    location=[richness["Latitude"].mean(), richness["Longitude"].mean()],
    zoom_start=zoom,
    tiles="cartodbpositron"
)

# Optional: cluster markers if you have many locations
marker_cluster = MarkerCluster().add_to(richness_map)

# Normalize marker size based on richness
max_richness = richness["Richness"].max()

for _, row in richness.iterrows():
    CircleMarker(
        location=(row["Latitude"], row["Longitude"]),
        radius=5 + 10 * (row["Richness"] / max_richness),  # Size scaled by richness
        color="blue",
        fill=True,
        fill_color="green",
        fill_opacity=0.6,
        popup=f"<b>{row['Location']}</b><br>Richness: {row['Richness']}",
    ).add_to(marker_cluster)


# Export the map to HTML
export_file_path = export_folder_path / f"{export_file_name}.html"
richness_map.save(export_file_path.absolute())

# Show the map
richness_map